<a href="https://colab.research.google.com/github/Jonathanpro/myaiblog/blob/master/_notebooks/2021-07-09-huggingface_ner_pytorch_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP -  Named Entity Recognition - Training
> A tutorial for training a named entity recognition with Huggingface & Pytroch.

- toc: true 
- badges: true
- comments: false
- categories: [jupyter, NLP, TPU]
- image: images/chart-preview.png

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!curl https://github.com/elenanereiss/Legal-Entity-Recognition/raw/master/data/dataset_courts.zip -L -o /content/raw.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   168  100   168    0     0    282      0 --:--:-- --:--:-- --:--:--   282
100 4289k  100 4289k    0     0  3338k      0  0:00:01  0:00:01 --:--:-- 19.2M


In [ ]:
!unzip /content/raw.zip -d /content/raw/

Archive:  /content/raw.zip
  inflating: /content/raw/bfh.conll  
  inflating: /content/raw/bgh.conll  
  inflating: /content/raw/bpatg.conll  
  inflating: /content/raw/bsg.conll  
  inflating: /content/raw/bverfg.conll  
  inflating: /content/raw/bverwg.conll  
  inflating: /content/raw/bag.conll  


In [ ]:
!pip install transformers

     |████████████████████████████████| 2.5MB 7.0MB/s 
     |████████████████████████████████| 901kB 34.3MB/s 
     |████████████████████████████████| 3.3MB 38.8MB/s 


In [ ]:
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

     |████████████████████████████████| 149.9MB 77kB/s 
     |████████████████████████████████| 61kB 3.1MB/s 
ERROR: earthengine-api 0.1.269 has requirement google-api-python-client<2,>=1.12.1, but you'll have google-api-python-client 1.8.0 which is incompatible.
  Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8


In [ ]:
from pathlib import Path
import re

def read_wnut(file_path):
    file_path = Path(file_path)

    raw_text = file_path.read_text().strip()
    raw_docs = re.split(r'\n\t?\n', raw_text)
    token_docs = []
    tag_docs = []
    for doc in raw_docs:
        tokens = []
        tags = []
        for line in doc.split('\n'):
            # print(line)
            token, tag = line.split()
            tokens.append(token)
            tags.append(tag)
        token_docs.append(tokens)
        tag_docs.append(tags)

    return token_docs, tag_docs

train_texts, train_tags = read_wnut('/content/raw/bag.conll')
val_texts, val_tags = read_wnut('/content/raw/bgh.conll')

In [ ]:
tags = train_tags + val_tags

In [ ]:
# TODO visualize with print

In [ ]:
unique_tags = set(tag for doc in tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [ ]:
from transformers import AutoTokenizer
MODEL_NAME = 'bert-base-german-cased' 
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
train_encodings = tokenizer(train_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
val_encodings = tokenizer(val_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)

In [ ]:
import numpy as np

def encode_tags(tags, encodings):
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        # create an empty array of -100
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)

        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())

    return encoded_labels

train_labels = encode_tags(train_tags, train_encodings)
val_labels = encode_tags(val_tags, val_encodings)

In [ ]:
import torch

class COURTS_Dataset_train(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_encodings.pop("offset_mapping") # we don't want to pass this to the model
val_encodings.pop("offset_mapping")
train_dataset = COURTS_Dataset_train(train_encodings, train_labels)
val_dataset = COURTS_Dataset_train(val_encodings, val_labels)

In [ ]:
from transformers import AutoModelForTokenClassification
model = AutoModelForTokenClassification.from_pretrained(MODEL_NAME, num_labels=len(unique_tags))

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-b

Set device to TPU

In [ ]:
import torch_xla
import torch_xla.core.xla_model as xm

device = xm.xla_device()

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=30,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=5000,
)

# Map model to TPU
model = model.to(device)
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 12791
  Num Epochs = 30
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 24000


Step,Training Loss
5000,0.058200
10000,0.003400
15000,0.001700
20000,0.000800


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-2000
Configuration saved in ./results/checkpoint-2000/config.json
Model weights saved in ./results/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-2500
Configuration saved in ./results/checkpoint-2500/config.json
Model weights saved in ./results/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-3000
Configuration saved in ./results/checkpoint-3

Step,Training Loss
5000,0.058200
10000,0.003400
15000,0.001700
20000,0.000800


Saving model checkpoint to ./results/checkpoint-22000
Configuration saved in ./results/checkpoint-22000/config.json
Model weights saved in ./results/checkpoint-22000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-22500
Configuration saved in ./results/checkpoint-22500/config.json
Model weights saved in ./results/checkpoint-22500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-23000
Configuration saved in ./results/checkpoint-23000/config.json
Model weights saved in ./results/checkpoint-23000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-23500
Configuration saved in ./results/checkpoint-23500/config.json
Model weights saved in ./results/checkpoint-23500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-24000
Configuration saved in ./results/checkpoint-24000/config.json
Model weights saved in ./results/checkpoint-24000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models

TrainOutput(global_step=24000, training_loss=0.013400553142031034, metrics={'train_runtime': 9714.0783, 'train_samples_per_second': 39.502, 'train_steps_per_second': 2.471, 'total_flos': 8.345190008162549e+16, 'train_loss': 0.013400553142031034, 'epoch': 30.0})

In [ ]:
trainer.save_model("/content/gdrive/MyDrive/NER_01_030_2021_07_08.bin")

Saving model checkpoint to /content/gdrive/MyDrive/NER_01_030_2021_07_08.bin
Configuration saved in /content/gdrive/MyDrive/NER_01_030_2021_07_08.bin/config.json
Model weights saved in /content/gdrive/MyDrive/NER_01_030_2021_07_08.bin/pytorch_model.bin


In [ ]:
class COURTS_Dataset_inf(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

In [ ]:
sentence = ' '.join(train_texts[0])
train_texts[0]

['Prozesskostenhilfe',
 '-',
 'Entschädigung',
 'für',
 'überlange',
 'Verfahrensdauer',
 '-',
 'Revisionsverfahren']

In [ ]:
def predict_text(text, train_tags=train_tags,  trainer=trainer, tokenizer=tokenizer):
    train_encodings = tokenizer([text], is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
    train_encodings.pop("offset_mapping") # we don't want to pass this to the model
    train_dataset = COURTS_Dataset_inf(train_encodings)
    result = trainer.predict(train_dataset)

    token_ids = tokenizer.encode(text[0], add_special_tokens=False)
    word_tag_dic = []
    preds = np.argmax(result.predictions, axis=2)
    x=1
    for word in text[0].split(' '):
        word_tag_dic.append([word, id2tag[preds[0][x]]])
        x=x+1
    return word_tag_dic

In [ ]:
word_tag_dic=predict_text([' '.join(train_texts[3])])

***** Running Prediction *****
  Num examples = 1
  Batch size = 64


In [ ]:
[' '.join(train_texts[3])]

['Die beabsichtigte Klage auf Entschädigung wegen der Dauer des Verfahrens vor dem Bundesarbeitsgericht zum Aktenzeichen - 8 AZR 418/15 - bietet keine hinreichende Aussicht auf Erfolg , § 114 Abs. 1 Satz 1 ZPO .']

In [ ]:
train_texts[3]

['Die',
 'beabsichtigte',
 'Klage',
 'auf',
 'Entschädigung',
 'wegen',
 'der',
 'Dauer',
 'des',
 'Verfahrens',
 'vor',
 'dem',
 'Bundesarbeitsgericht',
 'zum',
 'Aktenzeichen',
 '-',
 '8',
 'AZR',
 '418/15',
 '-',
 'bietet',
 'keine',
 'hinreichende',
 'Aussicht',
 'auf',
 'Erfolg',
 ',',
 '§',
 '114',
 'Abs.',
 '1',
 'Satz',
 '1',
 'ZPO',
 '.']

In [ ]:
train_tags[3]

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-RS',
 'I-RS',
 'I-RS',
 'I-RS',
 'I-RS',
 'I-RS',
 'I-RS',
 'I-RS',
 'I-RS',
 'I-RS',
 'I-RS',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-GS',
 'I-GS',
 'I-GS',
 'I-GS',
 'I-GS',
 'I-GS',
 'I-GS',
 'O']